In [43]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from os import getenv
import re

load_dotenv()
database_url = getenv("DATABASE_URL")

table_name = 'product_performance'
category = 'conversions'

data_types = {
    'ga:productSku': 'object',
    'ga:productName': 'object',
    'ga:productCategory': 'object',
    'ga:date': 'object',
    'ga:itemRevenue': 'float64',
    'ga:uniquePurchases': 'int64',
    'ga:itemQuantity': 'int64',
    'view_name': 'object'
}

df = pd.read_csv(f"data/{category}/{table_name}.csv", dtype=data_types)

for col in data_types.keys():
    if col not in df.columns:
        print(f'Column {col} not found in the dataset')
    assert col in df.columns

# df.columns = df.columns.str.replace('Unnamed: 0', 'id').replace('ga:', '')
df.columns = df.columns.str.replace('Unnamed: 0', 'id')
df.columns = df.columns.str.replace('ga:', '')
df.columns = df.columns.map(lambda x: re.sub(r'([A-Z])', r'_\1', x).lower())

df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')


print(f'{len(df)} records to upload...')


df.to_sql(table_name, database_url, if_exists='replace', chunksize=50000, index=False)

print('Done!')

9332 records to upload...
Done!
